In [1]:
# In[1]: Imports & load checkpoint
import torch
import pickle

# ←– EDIT THIS to point at your saved model.ckpt
checkpoint_path = "/home/azeez/Documents/Tri-MipRF/output/2025-05-12_18-38-47/Pipe_Big_Adapter/base_000_000_000/model.ckpt"

# load to CPU
ckpt = torch.load(checkpoint_path, map_location="cpu")
state_dict = ckpt["model"]

print(f"✅ Loaded '{checkpoint_path}' with {len(state_dict)} parameters.")

✅ Loaded '/home/azeez/Documents/Tri-MipRF/output/2025-05-12_18-38-47/Pipe_Big_Adapter/base_000_000_000/model.ckpt' with 10 parameters.


In [2]:
# In[2]: List all parameters and their shapes
print("All parameter shapes:")
for name, tensor in state_dict.items():
    print(f"  {name:60s} → {tuple(tensor.shape)}")

All parameter shapes:
  aabb                                                         → (6,)
  occ_level_vol                                                → ()
  field.encoding.fm                                            → (3, 512, 512, 16)
  field.direction_encoding.params                              → (0,)
  field.mlp_base.params                                        → (24576,)
  field.mlp_head.params                                        → (55296,)
  ray_sampler._roi_aabb                                        → (6,)
  ray_sampler._binary                                          → (128, 128, 128)
  ray_sampler.resolution                                       → (3,)
  ray_sampler.occs                                             → (2097152,)


In [3]:
# In[3]: Extract and save key components
weights = {}

# 1) TriMipEncoding feature maps ("fm")
fm_key = "field.encoding.fm"
weights["fm"] = state_dict[fm_key].numpy()
print(f"\nExtracted   fm   → shape {weights['fm'].shape}")

# 2) Geometric MLP (mlp_base)
base_prefix = "field.mlp_base"
base_keys = [k for k in state_dict if k.startswith(base_prefix)]
weights["mlp_base"] = {k: state_dict[k].numpy() for k in base_keys}
print(f"Extracted mlp_base → {len(base_keys)} tensors:")
for k in base_keys:
    print(" ", k, state_dict[k].shape)

# 3) View-direction MLP (mlp_head)
head_prefix = "field.mlp_head"
head_keys = [k for k in state_dict if k.startswith(head_prefix)]
weights["mlp_head"] = {k: state_dict[k].numpy() for k in head_keys}
print(f"\nExtracted mlp_head → {len(head_keys)} tensors:")
for k in head_keys:
    print(" ", k, state_dict[k].shape)

# 4) Directional encoding (tiny-cudann)
dir_prefix = "field.direction_encoding"
dir_keys = [k for k in state_dict if k.startswith(dir_prefix)]
weights["direction_encoding"] = {k: state_dict[k].numpy() for k in dir_keys}
print(f"\nExtracted direction_encoding → {len(dir_keys)} tensors:")
for k in dir_keys:
    print(" ", k, state_dict[k].shape)

# Save everything for later
out_path = "extracted_weights.pkl"
with open(out_path, "wb") as f:
    pickle.dump(weights, f)
print(f"\n✅ Saved all extracted weights to '{out_path}'")


Extracted   fm   → shape (3, 512, 512, 16)
Extracted mlp_base → 1 tensors:
  field.mlp_base.params torch.Size([24576])

Extracted mlp_head → 1 tensors:
  field.mlp_head.params torch.Size([55296])

Extracted direction_encoding → 1 tensors:
  field.direction_encoding.params torch.Size([0])

✅ Saved all extracted weights to 'extracted_weights.pkl'


In [4]:
from pathlib import Path
import pprint


BASE_OUTPUT_DIR = Path("/home/azeez/Documents/Tri-MipRF/output/2025-05-13_00-05-52")
COMPONENT_KEYS = {
    "fm": "field.encoding.fm",
    "mlp_base": "field.mlp_base",
    "mlp_head": "field.mlp_head",
    "direction_encoding": "field.direction_encoding",
}

# In[2]: Build mapping: run → object → variation → ckpt_path
runs = {}
for run_dir in sorted(BASE_OUTPUT_DIR.iterdir()):
    if not run_dir.is_dir():
        continue
    run_name = run_dir.name
    runs[run_name] = {}
    for obj_dir in sorted(run_dir.iterdir()):
        if not obj_dir.is_dir():
            continue
        obj_name = obj_dir.name
        runs[run_name][obj_name] = {}
        for var_dir in sorted(obj_dir.iterdir()):
            ckpt = var_dir / "model.ckpt"
            if ckpt.exists():
                runs[run_name][obj_name][var_dir.name] = ckpt

print("Discovered runs structure:")
pprint.pprint(runs)

Discovered runs structure:
{'Banana Box': {'base_000_000_000': {},
                'compound_090_000_090': {},
                'x_180_000_000': {},
                'y_000_180_000': {},
                'z_000_000_120': {},
                'z_000_000_240': {}},
 'BarrelWithStand': {'base_000_000_000': {},
                     'compound_090_000_090': {},
                     'x_180_000_000': {},
                     'y_000_180_000': {},
                     'z_000_000_120': {},
                     'z_000_000_240': {}},
 'Bed Room': {'base_000_000_000': {},
              'compound_090_000_090': {},
              'x_180_000_000': {},
              'y_000_180_000': {},
              'z_000_000_120': {},
              'z_000_000_240': {}},
 'BerryBush1': {'base_000_000_000': {},
                'compound_090_000_090': {},
                'x_180_000_000': {},
                'y_000_180_000': {},
                'z_000_000_120': {},
                'z_000_000_240': {}},
 'BvpDestro': {'base_00

In [5]:
# In[1]: Discover models → then import torch and load components

from pathlib import Path

# 1) Scan file system for objects & variations
ROOT_DIR = Path("/home/azeez/Documents/Tri-MipRF/output/2025-05-13_12-59-05")
models = {
    obj.name: {
        var.name: var / "model.ckpt"
        for var in obj.iterdir()
        if (var / "model.ckpt").exists()
    }
    for obj in ROOT_DIR.iterdir() if obj.is_dir()
}
print("Discovered models:")
for obj, vars_map in models.items():
    print(f"- {obj}: {list(vars_map.keys())}")

# 2) Import torch once scanning is done
import torch

COMPONENT_KEYS = {
    "fm": "field.encoding.fm",
    "mlp_base": "field.mlp_base",
    "mlp_head": "field.mlp_head",
    "direction_encoding": "field.direction_encoding",
}

# 3) Loader function
def load_components(ckpt_path):
    sd = torch.load(ckpt_path, map_location="cpu")["model"]
    comps = {}
    comps["fm"] = sd[COMPONENT_KEYS["fm"]].numpy()
    comps["mlp_base"] = {k: sd[k].numpy() for k in sd if k.startswith(COMPONENT_KEYS["mlp_base"])}
    comps["mlp_head"] = {k: sd[k].numpy() for k in sd if k.startswith(COMPONENT_KEYS["mlp_head"])}
    comps["direction_encoding"] = {k: sd[k].numpy() for k in sd if k.startswith(COMPONENT_KEYS["direction_encoding"])}
    return comps

# 4) Example load & print shapes
obj0 = next(iter(models))
var0 = next(iter(models[obj0]))
ckpt_path = models[obj0][var0]
print(f"\nLoading from: {ckpt_path}")
components = load_components(ckpt_path)

print("\nComponent shapes:")
for name, val in components.items():
    if isinstance(val, dict):
        print(f"{name}:")
        for k, arr in val.items():
            print(f"  {k:60s} → {arr.shape}")
    else:
        print(f"{name:20s} → {val.shape}")

Discovered models:
- Book_Stack3: ['y_000_180_000', 'base_000_000_000', 'z_000_000_240', 'z_000_000_120', 'compound_090_000_090', 'x_180_000_000']
- Chassi 2x2m: ['y_000_180_000', 'base_000_000_000', 'z_000_000_240', 'z_000_000_120', 'compound_090_000_090', 'x_180_000_000']
- flower_crown: ['y_000_180_000', 'base_000_000_000', 'z_000_000_240', 'z_000_000_120', 'compound_090_000_090', 'x_180_000_000']
- Build8: ['y_000_180_000', 'base_000_000_000', 'z_000_000_240', 'z_000_000_120', 'compound_090_000_090', 'x_180_000_000']
- Cafeteira: ['y_000_180_000', 'base_000_000_000', 'z_000_000_240', 'z_000_000_120', 'compound_090_000_090', 'x_180_000_000']
- UtilitySink: ['y_000_180_000', 'base_000_000_000', 'z_000_000_240', 'z_000_000_120', 'compound_090_000_090', 'x_180_000_000']
- gramofonebase: ['y_000_180_000', 'base_000_000_000', 'z_000_000_240', 'z_000_000_120', 'compound_090_000_090', 'x_180_000_000']
- Coffin_06_Lod0: ['y_000_180_000', 'base_000_000_000', 'z_000_000_240', 'z_000_000_120',

In [ ]:
print(components)